In [1]:
import pandas as pd
import numpy as np
import datetime
import networkx as nx

First of all we decompressed the '.gz' files and then added a row as a header ('from to time').

In [53]:
#load data
answers_to_questions = pd.read_table('answers_to_questions', sep=' ')
comments_to_answers = pd.read_table('comments_to_answers', sep=' ')
comments_to_questions = pd.read_table('comments_to_questions', sep=' ')

If you wanna work on a smaller range of time use the code below. For example let's just take the data from 2010.

In [13]:
#create timestamps for the first and last days of 2010
import time
start = '01/01/2010'
end = '31/12/2010'
start = int(time.mktime(datetime.datetime.strptime(start, "%d/%m/%Y").timetuple()))
end = int(time.mktime(datetime.datetime.strptime(end, "%d/%m/%Y").timetuple()))


#take only data from the first half of 2009
answers_to_questions = answers_to_questions[(answers_to_questions.time >= start) & (answers_to_questions.time <= end)]
comments_to_answers = comments_to_answers[(comments_to_answers.time >= start) & (comments_to_answers.time <= end)]
comments_to_questions = comments_to_questions[(comments_to_questions.time >= start) & (comments_to_questions.time <= end)]
#reset the index
answers_to_questions.reset_index(drop = True, inplace = True)
comments_to_answers.reset_index(drop = True, inplace = True)
comments_to_questions.reset_index(drop = True, inplace = True)

Let's deal with dates!

In [54]:
#convert timestamps
answers_to_questions['time'] = answers_to_questions['time'].apply(datetime.datetime.fromtimestamp).astype(str)
comments_to_answers['time'] = comments_to_answers['time'].apply(datetime.datetime.fromtimestamp).astype(str)
comments_to_questions['time'] = comments_to_questions['time'].apply(datetime.datetime.fromtimestamp).astype(str)

#let's just take year, month, day (in this order)
answers_to_questions['time'] = answers_to_questions['time'].str[:10]
comments_to_answers['time'] = comments_to_answers['time'].str[:10]
comments_to_questions['time'] = comments_to_questions['time'].str[:10]

In [66]:
answers_to_questions.head()

,from,to,time
0,9,8,2008-08-01
1,1,1,2008-08-01
2,13,1,2008-08-01
3,17,1,2008-08-01
4,48,2,2008-08-01


In [56]:
comments_to_answers.head()

,from,to,time
0,1,91,2008-09-06
1,3,91,2008-09-06
2,380,350,2008-09-06
3,4642,2257,2008-09-06
4,4642,1324220,2008-09-06


In [57]:
comments_to_questions.head()

,from,to,time
0,4550,4550,2008-09-06
1,242,184,2008-09-06
2,4213,4946,2008-09-07
3,91,91,2008-09-07
4,2658,1874,2008-09-07


Each dictionary represents a adjacency list. 

The dictionary *Graph* represent the merge of the 3 single graphs: for each node *u* there are all the nodes *v* , represented as dictionaries, which answered to a question of node *u*, commented an answer of node *u* or commented a question of node *u*. This graph is directed and self-connections are allowed. Each dictionary represented by the nodes *v* cointains a list of all the dates of the actions.

We can think at the dictionary *Graph* as am adjacency list. Moreover each link is weigthed, and the weight is equal to the number of actions taken, which can be calculated by taking the length of the list inside each dictionary *v*.

In [ ]:
answers_to_questions_graph = {}

for i in range(len(answers_to_questions)):
    if answers_to_questions['from'][i] not in answers_to_questions_graph:
        answers_to_questions_graph[answers_to_questions['from'][i]] = []
    answers_to_questions_graph[answers_to_questions['from'][i]].append((answers_to_questions.to[i], answers_to_questions.time[i]))
    
    
supp = {}
for key in answers_to_questions_graph.keys():
    for i in range(len(answers_to_questions_graph[key])):
        if answers_to_questions_graph[key][i][0] not in supp:
            supp[answers_to_questions_graph[key][i][0]] = []
        supp[answers_to_questions_graph[key][i][0]].append(answers_to_questions_graph[key][i][1])
    answers_to_questions_graph[key] = supp

In [ ]:
comments_to_answers_graph = {}

for i in range(len(comments_to_answers)):
    if comments_to_answers['from'][i] not in comments_to_answers_graph:
        comments_to_answers_graph[comments_to_answers['from'][i]] = []
    comments_to_answers_graph[comments_to_answers['from'][i]].append((comments_to_answers.to[i], comments_to_answers.time[i]))
    

supp = {}
for key in comments_to_answers_graph.keys():
    for i in range(len(comments_to_answers_graph[key])):
        if comments_to_answers_graph[key][i][0] not in supp:
            supp[comments_to_answers_graph[key][i][0]] = []
        supp[comments_to_answers_graph[key][i][0]].append(comments_to_answers_graph[key][i][1])
    comments_to_answers_graph[key] = supp

In [ ]:
comments_to_questions_graph = {}

for i in range(len(comments_to_questions)):
    if comments_to_questions['from'][i] not in comments_to_questions_graph:
        comments_to_questions_graph[comments_to_questions['from'][i]] = []
    comments_to_questions_graph[comments_to_questions['from'][i]].append((comments_to_questions.to[i], comments_to_questions.time[i]))
    

supp = {}
for key in comments_to_questions_graph.keys():
    for i in range(len(comments_to_questions_graph[key])):
        if comments_to_questions_graph[key][i][0] not in supp:
            supp[comments_to_questions_graph[key][i][0]] = []
        supp[comments_to_questions_graph[key][i][0]].append(comments_to_questions_graph[key][i][1])
    comments_to_questions_graph[key] = supp

In [ ]:
graph = {}

for i in range(len(answers_to_questions)):
    if answers_to_questions['from'][i] not in graph:
        graph[answers_to_questions['from'][i]] = []
    graph[answers_to_questions['from'][i]].append((answers_to_questions.to[i], answers_to_questions.time[i]))
    
for i in range(len(comments_to_answers)):
    if comments_to_answers['from'][i] not in graph:
        graph[comments_to_answers['from'][i]] = []
    graph[comments_to_answers['from'][i]].append((comments_to_answers.to[i], comments_to_answers.time[i]))
    
for i in range(len(comments_to_questions)):
    if comments_to_questions['from'][i] not in graph:
        graph[comments_to_questions['from'][i]] = []
    graph[comments_to_questions['from'][i]].append((comments_to_questions.to[i], comments_to_questions.time[i]))
    
    
    
supp = {}
for key in graph.keys():
    for i in range(len(graph[key])):
        if graph[key][i][0] not in supp:
            supp[graph[key][i][0]] = []
        supp[graph[key][i][0]].append(graph[key][i][1])
    graph[key] = supp